In [1]:
import csv
import collections
import numpy as np

In [2]:
test_data_dict=[]
with open('test_data.csv', 'r') as f:
    reader = csv.DictReader(f)
    for line in enumerate(reader):
        test_data_dict.append(line[1])

In [3]:
grouped_by_game = collections.defaultdict(list)
for line in test_data_dict:
    grouped_by_game[line['gameid']].append(line)
grouped_by_game = list(grouped_by_game.values())

for game in grouped_by_game:
    for play_dict in game:
        for key in play_dict:
            try:
                play_dict[key] = int(play_dict[key])
            except:
                pass
            
for game in grouped_by_game:
    game.sort(key = lambda x: -int(x['time']))

In [21]:
processed_train_data = []
for i, game in enumerate(grouped_by_game):
    first_team_yards=0
    first_team_plays=0
    second_team_yards=0
    second_team_plays=0
    time_per_play = []
    team = ''
    alph_first_team = ''
    team_at_beg_fourth = ''
    beg_fourth = True
    
    for i in range(len(game)-1):
        play_dict = game[i]
        alph_first_team = play_dict['off'] if play_dict['off']<play_dict['def'] else play_dict['def']
        next_play_dict = game[i+1]
        if next_play_dict['down'] is not None and play_dict['down'] is not None and play_dict['ydline'] !='' and next_play_dict['ydline'] !='' and next_play_dict['off']==play_dict['off'] and next_play_dict['description'].find('punt')==-1 and play_dict['time']>=900:
            progress = play_dict['ydline']-next_play_dict['ydline']
            if play_dict['off']==alph_first_team:
                first_team_yards+=progress
                first_team_plays+=1
            else:
                second_team_yards+=progress
                second_team_plays+=1
        if (beg_fourth and play_dict['time']<900):
            beg_fourth = False
            team_at_beg_fourth = play_dict['off']
        
    off_team_avg_yards = (first_team_yards/first_team_plays) if team_at_beg_fourth==alph_first_team else (second_team_yards/second_team_plays)
    def_team_avg_yards = (second_team_yards/second_team_plays) if team_at_beg_fourth==alph_first_team else (first_team_yards/first_team_plays)
    
    final_diff = 0
    third_diff = 0
    yard_line = 0
    togo = 0
    down = 0
    off_avg = 0
    def_avg = 0
    beg_fourth = True
    for play in game: 
        if (play['time']>=900):
#             if play['offscore'] and play['defscore']
            third_diff = play['offscore']-play['defscore'] if team_at_beg_fourth==play['off'] else -(play['offscore']-play['defscore'])
    
        if (beg_fourth and play['time']<=900):
            if play['down']!='' and play['togo']!='':
                yard_line = play['ydline']
                down= play['down']
                togo = play['togo']
            else:
                down=-1
                togo=-1
                down=-1
            beg_fourth=False
            
        if (play['time']>=0):
            final_diff = play['offscore']-play['defscore'] if team_at_beg_fourth==play['off'] else -(play['offscore']-play['defscore'])
            
    processed_train_data.append((yard_line, togo, down, third_diff, final_diff, off_team_avg_yards, def_team_avg_yards))

In [25]:
processed_train_data[0]

(18, 10, 1, -6, -6, 5.590909090909091, 4.0)

In [26]:
processed_train_data[3]

(18, 9, 2, 10, 10, 3.0638297872340425, 4.605263157894737)

In [27]:
features = ['yard_line', 'togo', 'down', 'third_diff', 'final_diff', 'off_team_avg_yards', 'def_team_avg_yards']

In [28]:
with open('formatted_test.csv','w') as out:
    writer = csv.writer(out)
    writer.writerow(features)
    for datum in processed_train_data:
        writer.writerow(datum)

In [12]:
processed_train_data[12]

(79, 10, 1, 21, 18, 4.142857142857143, 5.473684210526316)

In [14]:
grouped_by_game[0]

[{'def': 'Rams',
  'defscore': 0,
  'description': '(15:00) R.Wilson pass short right to S.Rice to SEA 37 for 17 yards (C.Finnegan).',
  'down': 1,
  'gameid': '20120930_SEA@STL',
  'off': 'Seahawks',
  'offscore': 0,
  'time': 3600,
  'togo': 10,
  'ydline': 80},
 {'def': 'Rams',
  'defscore': 0,
  'description': '(14:23) R.Wilson scrambles right end to SEA 45 for 8 yards (R.McIntosh).',
  'down': 1,
  'gameid': '20120930_SEA@STL',
  'off': 'Seahawks',
  'offscore': 0,
  'time': 3563,
  'togo': 10,
  'ydline': 63},
 {'def': 'Rams',
  'defscore': 0,
  'description': '(14:04) M.Lynch left tackle to SEA 49 for 4 yards (J.Laurinaitis).',
  'down': 2,
  'gameid': '20120930_SEA@STL',
  'off': 'Seahawks',
  'offscore': 0,
  'time': 3544,
  'togo': 2,
  'ydline': 55},
 {'def': 'Rams',
  'defscore': 0,
  'description': '(13:29) M.Lynch left tackle to SL 48 for 3 yards (W.Hayes).',
  'down': 1,
  'gameid': '20120930_SEA@STL',
  'off': 'Seahawks',
  'offscore': 0,
  'time': 3509,
  'togo': 10,
 

In [15]:
processed_train_data[0]

(80, 10, 1, -6, -6, 5.590909090909091, 4.0)